In [7]:
import asyncio
import random
from functools import wraps

def async_retry(max_attempts=3, base_delay=1, backoff_factor=2):
    """
    Decorator that retries async functions with exponential backoff.
    
    Args:
        max_attempts: Maximum number of attempts
        base_delay: Initial delay between retries (seconds)
        backoff_factor: Multiply delay by this factor after each retry
    
    Example:
        @async_retry(max_attempts=3, base_delay=1, backoff_factor=2)
        async def unreliable_function():
            # Will retry with delays: 1s, 2s, 4s
            pass
    """
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            for attempt in range(max_attempts):
                try:
                    print(f"[Attempt {attempt + 1}/{max_attempts}]")
                    result = await func(*args, **kwargs)
                    return result
                except Exception as e:
                    if attempt == max_attempts - 1:
                        print(f"[All {max_attempts} attempts failed]")
                        raise
                    delay = base_delay * (backoff_factor ** attempt)
                    print(f"[Retrying in {delay}s...]")
                    await asyncio.sleep(delay)
        return wrapper
    return decorator


# Flaky API simulator
@async_retry(max_attempts=5, base_delay=0.5, backoff_factor=2)
async def flaky_api_call(success_rate=0.3):
    """
    Simulates an unreliable API call.
    
    Args:
        success_rate: Probability of success (0.0 to 1.0)
    """
    print(f"  Attempting API call...")
    await asyncio.sleep(0.1)  # Simulate network delay
    
    if random.random() < success_rate:
        print("Success!")
        return "API response data"
    else:
        print("Failed!")
        raise ConnectionError("API temporarily unavailable")


async def test_retry():
    print("Test 1: Flaky API (30% success rate)")
    try:
        result = await flaky_api_call(success_rate=0.3)
        print(f"Final result: {result}\n")
    except ConnectionError as e:
        print(f"All retries failed: {e}\n")
    
    print("Test 2: Very flaky API (10% success rate)")
    try:
        result = await flaky_api_call(success_rate=0.1)
        print(f"Final result: {result}\n")
    except ConnectionError as e:
        print(f"All retries failed: {e}\n")


# Run test
await test_retry()

Test 1: Flaky API (30% success rate)
[Attempt 1/5]
  Attempting API call...
Failed!
[Retrying in 0.5s...]
[Attempt 2/5]
  Attempting API call...
Failed!
[Retrying in 1.0s...]
[Attempt 3/5]
  Attempting API call...
Success!
Final result: API response data

Test 2: Very flaky API (10% success rate)
[Attempt 1/5]
  Attempting API call...
Failed!
[Retrying in 0.5s...]
[Attempt 2/5]
  Attempting API call...
Failed!
[Retrying in 1.0s...]
[Attempt 3/5]
  Attempting API call...
Failed!
[Retrying in 2.0s...]
[Attempt 4/5]
  Attempting API call...
Failed!
[Retrying in 4.0s...]
[Attempt 5/5]
  Attempting API call...
Failed!
[All 5 attempts failed]
All retries failed: API temporarily unavailable



In [12]:
def batch(iterable, batch_size):
    """
    Generator that yields items in batches.
    
    Args:
        iterable: Any iterable (list, generator, etc.)
        batch_size: Number of items per batch
    
    Yields:
        Lists of items with length = batch_size (last batch may be smaller)
    
    Example:
        >>> list(batch([1, 2, 3, 4, 5, 6, 7], 3))
        [[1, 2, 3], [4, 5, 6], [7]]
        
        >>> list(batch(range(10), 4))
        [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]]
    """
    iterater = iter(iterable)
    while True:
        res = [] 
        try:
            for _ in range(batch_size):
                res.append(next(iterater))
        except StopIteration:
            if res:
                yield res 
            break 
        yield res 

# Test cases
print(list(batch([1, 2, 3, 4, 5, 6, 7], 3)))
# [[1, 2, 3], [4, 5, 6], [7]]

print(list(batch(range(10), 4)))
# [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]]

print(list(batch("ABCDEFGH", 2)))
# [['A', 'B'], ['C', 'D'], ['E', 'F'], ['G', 'H']]

[[1, 2, 3], [4, 5, 6], [7]]
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]]
[['A', 'B'], ['C', 'D'], ['E', 'F'], ['G', 'H']]


In [ ]:

def cache_with_log(func):
    """
    Cache decorator that logs all activity.
    Should work with any function signature.
    """
    cache = {} 
    def wrapper(*args, **kwargs):
        key = (args, frozenset(kwargs.items()))
        print(args, kwargs, *args, kwargs)
        # **kwargs不能直接放在print()里，py会把它当作kwargs传给print，而print没有greeting这个key所以报错, 直接用dic即kwargs即可
        if key in cache:
            print(f'Cache HIT for args= {args} kwargs= {kwargs}')
            return cache[key]
        else:
            print(f'Cache HIT for args= {args} kwargs= {kwargs}')
            res = func(*args, **kwargs)
            cache[key] = res 
            return res
    return wrapper 

# Test cases
@cache_with_log
def add(a, b):
    """Simple function with positional args"""
    return a + b

@cache_with_log
def greet(name, greeting="Hello"):
    """Function with keyword args"""
    return f"{greeting}, {name}!"

@cache_with_log
def calculate(x, y, operation="add"):
    """Function with mixed args"""
    if operation == "add":
        return x + y
    elif operation == "multiply":
        return x * y


# Run tests
print("=== Test 1: Simple function ===")
print(add(2, 3))      # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(add(2, 3))      # Should log: Cache HIT for args={args}, kwargs={kwargs}
print(add(4, 5))      # Should log: Cache MISS for args={args}, kwargs={kwargs}

print("\n=== Test 2: Function with kwargs ===")
print(greet("Alice"))                    # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(greet("Alice"))                    # Should log: Cache HIT for args={args}, kwargs={kwargs}
print(greet("Bob", greeting="Hi"))       # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(greet("Bob", greeting="Hi"))       # Should log: Cache HIT for args={args}, kwargs={kwargs}

print("\n=== Test 3: Mixed args ===")
print(calculate(3, 4))                   # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(calculate(3, 4, operation="add"))  # Should log: Cache HIT for args={args}, kwargs={kwargs}
print(calculate(3, 4, operation="multiply"))  # Should log: Cache MISS for args={args}, kwargs={kwargs}

=== Test 1: Simple function ===
(2, 3) {} 2 3 {}
Cache HIT for args= (2, 3) kwargs= {}
5
(2, 3) {} 2 3 {}
Cache HIT for args= (2, 3) kwargs= {}
5
(4, 5) {} 4 5 {}
Cache HIT for args= (4, 5) kwargs= {}
9

=== Test 2: Function with kwargs ===
('Alice',) {} Alice {}
Cache HIT for args= ('Alice',) kwargs= {}
Hello, Alice!
('Alice',) {} Alice {}
Cache HIT for args= ('Alice',) kwargs= {}
Hello, Alice!
('Bob',) {'greeting': 'Hi'} Bob {'greeting': 'Hi'}
Cache HIT for args= ('Bob',) kwargs= {'greeting': 'Hi'}
Hi, Bob!
('Bob',) {'greeting': 'Hi'} Bob {'greeting': 'Hi'}
Cache HIT for args= ('Bob',) kwargs= {'greeting': 'Hi'}
Hi, Bob!

=== Test 3: Mixed args ===
(3, 4) {} 3 4 {}
Cache HIT for args= (3, 4) kwargs= {}
7
(3, 4) {'operation': 'add'} 3 4 {'operation': 'add'}
Cache HIT for args= (3, 4) kwargs= {'operation': 'add'}
7
(3, 4) {'operation': 'multiply'} 3 4 {'operation': 'multiply'}
Cache HIT for args= (3, 4) kwargs= {'operation': 'multiply'}
12
